# Wrapping dependent libraries

**StructureAnalysis** is a set of libraries that include statistical models for the analysis of structured data (mainly sequences and tree-structured data):

* **StatTool** is a library containing classes concerning parametric  modeling of univariate and multivariate data.

* **SequenceAnalysis** is a library containing statistical functions and classes for markovian models (e.g., hidden variable-order Markov and hidden semi-Markov models) and multiple change-point models for sequences.
  The **SequenceAnalysis** library depends on the **StatTool** library.

These libraries have been extensively used for the identification and characterization of developmental patterns in plants from the tissular to the whole plant scale.
Previously interfaced with *AML* (a home-made, domain-specific programming language), some work has been done to switch to *Python*.
Nevertheless, the complexity of writing wrappers with **Boost.Python** limited the number of available components in *Python* in comparison to *AML*.
One advantage of having a statistical library written in *C++* available in *Python* is that developers can benefit from all *Python* packages.
As illustrated with the following figures this is particularly useful for providing visualizations for model quality assessment using -- for example -- the **Matplotlib** *Python* package.

In [ ]:
import matplotlib
%matplotlib nbagg

## Preamble

The code source is available in the `StructureAnalysis` directory.

In [ ]:
!pygmentize StructureAnalysis/stat_tool/src/cpp/stat_tools.h

This directory already contains wrappers, we therefore need to remove them.

In [ ]:
from path import path
srcdir = dict()
for library in ['stat_tool', 'sequence_analysis']:
    srcdir[library] = path('StructureAnalysis')/library/'src'/'py'
    for wrapper in srcdir[library].walkfiles('*.cpp'):
        wrapper.unlink()
    for wrapper in srcdir[library].walkfiles('*.h'):
        wrapper.unlink()
    wrapper = srcdir[library]/library/'_' + library + '.py'
    if wrapper.exists():
        wrapper.unlink()
    srcdir[library] = srcdir[library].parent.parent

Then, we need to install and compile the *C++* libraries.
This is done using available **Conda** recipes.

In [ ]:
!conda build StructureAnalysis/stat_tool/conda/libstat_tool -c statiskit -c conda-forge
!conda install libstat_tool --use-local -c statiskit -c conda-forge
!conda build StructureAnalysis/sequence_analysis/conda/libsequence_analysis -c statiskit -c conda-forge
!conda install libsequence_analysis --use-local -c statiskit -c conda-forge

## The **StatTool** library

### Wrapping the  *C++* library
Once these preliminaries done, we can proceed to the actual generation of wrappers for the **StatTool** *C++* library.
For this, we import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

In [ ]:
import autowig
asg = autowig.AbstractSemanticGraph()

Then, we parse headers with relevant compilation flags.

In [ ]:
%%time
import sys
prefix = path(sys.prefix)
autowig.parser.plugin = 'pyclanglite'
asg = autowig.parser(asg, (prefix/'include'/'stat_tool').walkfiles('*.h*'),
                          ['-x', 'c++', '-std=c++11',  '-I' + str((prefix/'include').abspath())],
                          silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation could be suitable.
Nevertheless some **AutoWIG** limitations (**AutoWIG** doesn't have a complete knowledge concering copyable classes) and the requirement of classes defined in the standard *C++* library lead us to implement a new `controller`.

In [ ]:
def stat_tool_controller(asg):
    for noncopyable in ['class ::std::basic_streambuf< char, struct ::std::char_traits< char > >',
                        'class ::std::codecvt< char, char, __mbstate_t >',
                        'class ::std::basic_filebuf< char, struct ::std::char_traits< char > >',
                        'class ::std::locale::facet',
                        'class ::std::locale::id',
                        'class ::std::ctype< char >',
                        'class ::std::ios_base',
                        'class ::std::basic_istream< char, struct ::std::char_traits< char > >',
                        'class ::std::basic_ifstream< char, struct ::std::char_traits< char > >',
                        'class ::std::basic_ostream< char, struct ::std::char_traits< char > >',
                        'class ::std::basic_ostringstream< char, struct ::std::char_traits< char >, class ::std::allocator< char > >',
                        'class ::std::num_get< char, class ::std::istreambuf_iterator< char, struct ::std::char_traits< char > > >',
                        'class ::std::basic_ios< char, struct ::std::char_traits< char > >',
                        'class ::std::basic_stringbuf< char, struct ::std::char_traits< char >, class ::std::allocator< char > >',
                        'class ::std::num_put< char, class ::std::ostreambuf_iterator< char, struct ::std::char_traits< char > > >']:
        asg[noncopyable].is_copyable = False
    for cls in asg.classes():
        for fld in cls.fields(access='public'):
            if fld.qualified_type.unqualified_type.globalname == 'class ::std::locale::id':
                fld.boost_python_export = False
    for specialization in asg['class ::std::reverse_iterator'].specializations():
        specialization.boost_python_export = False
    asg['::std::ios_base::openmode'].qualified_type.boost_python_export = True
    return asg

This `controller` is then dynamically registered and used on the ASG.

In [ ]:
%%time
autowig.controller['stat_tool'] = stat_tool_controller
autowig.controller.plugin = 'stat_tool'
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
%%time
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = 'StructureAnalysis/stat_tool/src/py/_stat_tool.cpp',
                             decorator = 'StructureAnalysis/stat_tool/src/py/structure_analysis/stat_tool/_stat_tool.py',
                             prefix = 'wrapper_')

But we also require to the use the default `boost_python` `generator` implementation in order to wrap the `std::ostringstream` typedef.

In [ ]:
#%%time
autowig.generator.plugin = 'boost_python'
wrappers = autowig.generator(asg,
                             [asg['::std::ostringstream'],
                              asg['::std::ios_base::openmode']],
                             module = wrappers.globalname,
                             prefix = 'wrapper_')

The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [ ]:
%%time
wrappers.write()

Here is an example of the generated wrappers.

In [ ]:
!pygmentize StructureAnalysis/stat_tool/src/py/_stat_tool.cpp

In order to wrap a *C++* library, that will be used as a dependency by other libraries, the user needs to save the ASG resulting from the wrapping process.
We therefore use the **pickle** *Python* package for serializing the **StatTool** ASG in the `'StructureAnalysis/stat_tool/ASG.pkl'` file.

In [ ]:
import pickle
with open('StructureAnalysis/stat_tool/ASG.pkl', 'w') as filehandler:
    pickle.dump(asg, filehandler)

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [ ]:
!conda build StructureAnalysis/stat_tool/conda/python-stat_tool -c statiskit -c conda-forge
!conda install python-stat_tool --use-local -c statiskit -c conda-forge

### Using the *C++* library *Python* bindings

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
from structure_analysis import stat_tool
%reload_ext structure_analysis.stat_tool.mplotlib
%reload_ext structure_analysis.stat_tool.aml

The data  (`meri0`) consists of the number of elongated organs of $424$ wild cherry tree shoots (*Prunus avium*).
These shoots were sampled in different architectural positions (from the trunk to peripheral positions of the trees) and were representative of the full range of growth potential.
The proximal part of a shoot always consists of preformed organs i.e. organs contained in the winter bud (preformed organs are differentiated at the end of the preceding growing season and elongated at the beginning of the current growing season).
This preformed part may be followed by a neoformed part consisting of organs differentiated and elongated during the current growing season.

In [ ]:
meri1 = stat_tool.Histogram("StructureAnalysis/stat_tool/share/data/meri1.his")
meri2 = stat_tool.Histogram("StructureAnalysis/stat_tool/share/data/meri2.his")
meri3 = stat_tool.Histogram("StructureAnalysis/stat_tool/share/data/meri3.his")
meri4 = stat_tool.Histogram("StructureAnalysis/stat_tool/share/data/meri4.his")
meri5 = stat_tool.Histogram("StructureAnalysis/stat_tool/share/data/meri5.his")
meri0 = stat_tool.Merge(meri1, meri2, meri3, meri4, meri5)

We estimated binomial mixture models on the basis of this data.
The number of components ($2$) was selected using the bayesian information criterion with models ranging from $1$ up to $4$ components.

In [ ]:
mixt = stat_tool.MixtureEstimation(meri0, 1, 4, "BINOMIAL")

Further investigations can be performed in order to asses the quality of the mixture model with $2$ components.
For instance, we considered here the visualization of various probability functions.

In [ ]:
mix.plot()

As illustrated herabove the overall fitness of the model is good and:

* The first component corresponds to entirely preformed shoots.
* The second component to mixed shoots consisting of a preformed part followed by a neoformed part.

## The **SequenceAnalysis** library

### Wrapping the  *C++* library
Once the wrapping of the **StatTool** library is performed, we can proceed to the actual generation of wrappers for the **SequenceAnalysis** *C++* library.
In order to wrap a *C++* library that has dependencies, the user need to combine the ASGs resulting from the wrapping of its dependencies before performing its own wrapping.
For this, we create an empty Abstract Semantic Graph (ASG).

In [ ]:
asg = autowig.AbstractSemanticGraph()

Then, we use the **pickle** *Python* package for de-serializing the **StatTool** ASG and merge it in the current ASG.

In [ ]:
%%time
with open('StructureAnalysis/stat_tool/ASG.pkl', 'r') as filehandler:
    asg.merge(pickle.load(filehandler))

Then, we parse headers with relevant compilation flags.

In [ ]:
%%time
asg = autowig.parser(asg, (prefix/'include'/'sequence_analysis').walkfiles('*.h*'),
                          ['-x', 'c++', '-std=c++11',  '-I' + str((prefix/'include').abspath())],
                          silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is suitable.

In [ ]:
%%time
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
%%time
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = 'StructureAnalysis/sequence_analysis/src/py/_sequence_analysis.cpp',
                             decorator = 'StructureAnalysis/sequence_analysis/src/py/structure_analysis/sequence_analysis/_sequence_analysis.py',
                             prefix = 'wrapper_')

The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [ ]:
%%time
wrappers.write()

Here is an example of the generated wrappers.

In [ ]:
!pygmentize StructureAnalysis/sequence_analysis/src/py/_sequence_analysis.cpp'

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [ ]:
!conda build StructureAnalysis/sequence_analysis/conda/python-sequence_analysis -c statiskit -c conda-forge
!conda install python-sequence_analysis --use-local -c statiskit -c conda-forge

### Using the *C++* library *Python* bindings

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
from structure_analysis import sequence_analysis

The data (`seq`) consist of $4050$ measurements of the nuclear-magnetic response of underground rocks.
The data were obtained by lowering a probe into a bore-hole.
Measurements were taken at discrete time points by the probe as it was lowered through the hole.
The underlying signal is roughly piecewise constant, with each constant segment relating to a single rock type that has constant physical properties.
The change points in the signal occur each time a new rock type is encountered.
Outliers were removed before the data were analyzed.

In [ ]:
seq = sequence_analysis.Sequences("StructureAnalysis/sequence_analysis/share/data/well_log_filtered_indexed.seq")

We estimated Gaussian change in the mean and variance models on the basis of the well-log filtered data.
The number of segments ($16$) was selected using the slope heuristic with a slope estimated using log-likelihood of overparametrized models ranging from $30$ up to $80$ change points.

In [ ]:
seq.segmentation(0, 80, "Gaussian", min_nb_segment=30)